In [6]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
ls(mturk_path)

['../mturk/results/Batch_4258572_batch_results.csv:Zone.Identifier',
 '../mturk/results/Batch_4258572_batch_results.csv']

In [8]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
batch = 4258572
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,events,feedback,id,status
AssignmentId,,,,,,,
3SUWZRL0M48PRMNSEXLSDEW4JAA6EJ,3O2Y2UIUDVPM9VBJLAUYGVQZ34IFKE,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,NaN,NaN,NaN,NaN
"cut into 1- cubes""""}}","auce""""}}","{""""label"""": """"servings butter-flavored cookin...","{""""label"""": """"chili without beans""""","""""value"""": {""""IsRIgRFPW"""": """"sweet onions""""}}","{""""label"""": """"italian bread""""","""""value"""": {""""I7HcvcJwJ"""": """"italian bread""""}}","{""""label"""": """"recipe pastry for double-crust ..."
3YDTZAI2W3BCSYP2JLXY9WTWYA841L,3MNJFORX9GZJRCQGENQSE67H6EN5FJ,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,NaN,NaN,NaN,NaN
"cut into 1- cubes""""}}","auce""""}}","{""""label"""": """"servings butter-flavored cookin...","{""""label"""": """"chili without beans""""","""""value"""": {""""IsRIgRFPW"""": """"sweet onions""""}}","{""""label"""": """"italian bread""""","""""value"""": {""""I7HcvcJwJ"""": """"italian bread""""}}","{""""label"""": """"recipe pastry for double-crust ..."
3Z4AIRP3CC83Q0DC3WZ367QURZJ1XR,3JGHED38FIM4E7LU76XD1BST3V87YX,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A3KU8DC7HJOGCM,NaN,NaN,NaN,NaN
"cut into 1- cubes""""}}","auce""""}}","{""""label"""": """"servings butter-flavored cookin...","{""""label"""": """"chili without beans""""","""""value"""": {""""IsRIgRFPW"""": """"sweet onions""""}}","{""""label"""": """"italian bread""""","""""value"""": {""""I7HcvcJwJ"""": """"italian bread""""}}","{""""label"""": """"recipe pastry for double-crust ..."
3OS46CRSLLU6TX9E9NA2DIKQJYD6VB,3J94SKDELNKMTSCKNHV2Q64WU17D5G,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,AX8EX2QI5HIQQ,NaN,NaN,NaN,NaN
"cut into 1- cubes""""}}","auce""""}}","{""""label"""": """"servings butter-flavored cookin...","{""""label"""": """"chili without beans""""","""""value"""": {""""IsRIgRFPW"""": """"sweet onions""""}}","{""""label"""": """"italian bread""""","""""value"""": {""""I7HcvcJwJ"""": """"italian bread""""}}","{""""label"""": """"recipe pastry for double-crust ..."
308Q0PEVBE8OPARC8NEVI3LY1ZQI96,375VMB7D5OEY5G7ZU1EWFF81ZNDIDV,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,AORHXBTOCXFUK,NaN,NaN,NaN,NaN


In [5]:
assignment="  3OS46CRSLLU6TX9E9NA2DIKQJYD6VB".strip()
idx = df.loc[assignment, "id"]
status = df.loc[assignment, "status"]
display_html(f'<a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">{idx} - {status}</a>')

In [33]:
str(next(mturk_path.rglob("*4258572*.csv")))

'../mturk/results/Batch_4258572_batch_results.csv'